# Logistic Regression Proje

Machine Learning dünyasında çok ünlü bir problem var: Titanic.

Burada amaç Titanic gemisinde yer alan hangi yolcuların hayatta kaldığını tahmin etmek.

Titanic - Machine Learning from Disaster

https://www.kaggle.com/c/titanic/overview

### 1. Kütüphaneler

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

: 

### 2. Data Seti

**Önemli Not:** Kaggle yarışmalarında bazen data,
* train data
* test data
olarak ayrı verilir.

Buradaki test data bizim model oluştururken kullanacağımız test data değildir.

Bu test data Kaggle'in yarışma sonuçları test etmek için kullandığı datadır.

Dolayısı ile bu test data içinde sonuç değişkeni yani gerçek label değeri (grand truth) bulunmaz.

O yüzden biz kendi model doğruluğumuzu ölçerken o test datayı kullanamyız.

Biz yine train_test_split ile kendi test datamızı oluşturacağız.

In [ ]:
# CSV data'yı bir DataFrame içine al
train_df = pd.read_csv("data/train.csv")

In [ ]:
# train data
train_df.head()

### 3. Veri Kalitesi & Eksik Değerler (Missing Values)

Veri içinde her zaman eksik değerler bulunabilir.

Bu eksik değerleri yönetmenin çeşitli yolları vardır:
1. Eksik değer içeren satırları tamamen veri setinden çıkarmak
2. Eksik değerleri uygun değerler ile doldumak:
    * Mean (ortalama)
    * Median (Ortanca)

In [ ]:
# train data içindeki eksik değerlere bakalım

train_df.isnull().sum()

#### 3.1 Age - Eksik Değerler

In [ ]:
# Age (yaş) için eksik değerlerin oranı

train_df['Age'].isnull().sum() / train_df.shape[0] * 100

Yolcuların yaklaşık %20'sinin yaş (Age) değerleri eksik.

In [ ]:
ax = train_df['Age'].hist(bins=15, density=True, stacked=True, alpha=0.7)

train_df['Age'].plot(kind='density')

ax.set(xlabel='Age')
plt.xlim(0, 90)
plt.grid()
plt.show()

In [ ]:
# mean -> ortalama
# skipna -> eksik verileri es geç (skip)

train_df['Age'].mean(skipna=True)

In [ ]:
# median -> ortanca
# skipna -> eksik verileri es geç (skip)

train_df['Age'].median(skipna=True)

#### 3.2 Cabin - Eksik Değerler

In [ ]:
# Cabin için eksik değerler oranı

train_df['Cabin'].isnull().sum() / train_df.shape[0] * 100

Yolcuların %77'sinin Cabin numarası bilgisi eksik.

Burada kritik bir karar vermek gerekiyor.

Eldeki %23 ile %77'ün değerlerini mi doldurmalıyız yoksa bu sütunu tamamen çıkarmalı mıyız?

**Karar:** %77'lik bir eksik veri oranı çok yüksek olduğu için en doğrusu bu sütunu çıkarmak olacak.

#### 3.3 Embarked - Eksik Değerler

Embark -> gemiye binmek

In [ ]:
# embarked (gemiye binme) için eksik değer oranı

train_df['Embarked'].isnull().sum() / train_df.shape[0] * 100

%0.2 gibi (sadece 2 adet aslında) çok düşük bir oran o yüzden bu sütundaki eksik dataları dolduracağız.

In [ ]:
# yolcuların nereden bindiklerine bakalım

print('Yolcuların hangi limandan bindikleri % olarak: (C = Cherbourg, Q = Queenstown, S = Southampton):')
print(train_df['Embarked'].value_counts() / train_df.shape[0] * 100)

In [ ]:
# grafik ile de görelim

sns.countplot(x='Embarked', data=train_df, palette='Set1')
plt.show()

Eksik değerleri ne ile dolduracağımıza karar verirken burada (Embarked) Mean, Median vs kullanamayız,

Çünkü liman bilgisi kategorik bir bilgi.

O zaman ne yapacağız?

In [ ]:
# en fazla binilen limanı bulalım -> idmax()

print('En fazla binilen liman: ', train_df['Embarked'].value_counts().idxmax())

En fazla binilen liman uzakara S olduğu için biz de eksik dataları S olarak kabul edeceğiz.

#### 3.4 Nihai Karar ve Uygulama

Eksik değer içeren sütunlar için nihai karar:
* Age -> Median yöntemi ile eksik değerleri dolduracağız (median = 28)
* Embarked -> eksik değerleri 'S' olarak dolduracağız
* Cabin -> çok fazla (%77) eksik değer olduğu için bu sütunu çıkaracağız

In [ ]:
# önce orijinal verimizi kopyalayalım

train_data = train_df.copy()

In [ ]:
# Age -> eksikleri median ile dolduralım -> fillna()

train_data["Age"].fillna(train_df["Age"].median(skipna=True), inplace=True)

In [ ]:
# Embarked -> eksikleri 'S' ile dolduralım

train_data["Embarked"].fillna(train_df['Embarked'].value_counts().idxmax(), inplace=True)

In [ ]:
# Cabin -> bu sütunu çıkaralım -> drop

train_data.drop('Cabin', axis=1, inplace=True)

In [ ]:
# şimdi bu ayarlamalardan sonra train_data'da eksik veri var mı bakalım

train_data.isnull().sum()

In [ ]:
train_data.head(10)

#### 3.5 Fazladan olan (Gereksiz) Değişken Analizi

Birbiri ile ilgili olan iki sütun var: SibSp, Parch

**SibSp**: # of siblings / spouses aboard the Titanic

çocuk adedi (çift, karı-koca, için)

**Parch**: # of parents / children aboard the Titanic

çocuklar için ebeveyn sayısı

Gördüğünüz gibi, iki değişken de birbiri ile çok ilgili ve aralında yüksek oranda korelasyon var.

Bu iki değişkeni tek bir değişken altında toplamak daha sağlıklı olacak: **Yanlız mı seyahat ediyor**

Değişkenimizin adı **YalnizSeyahat** olsun. Ve bu kategorik bir değişken olacak. 0 veya 1.

In [ ]:
# SibSp ve Parch'a bakıp eğer ikisinin toplamı sıfır'dan büyükse o zaman yanlız seyahat etmiyordur -> 0
# diyeceğiz
# eğer toplamları sıfır ise o zaman yanlız seyahat ediyordur -> 1

train_data['YalnizSeyahat'] = np.where((train_data["SibSp"] + train_data["Parch"]) > 0, 0, 1)

In [ ]:
# SibSp ve Parch sütunlarını atalım

train_data.drop('SibSp', axis=1, inplace=True)

train_data.drop('Parch', axis=1, inplace=True)

In [ ]:
train_data.head()

#### 3.6 Kategorik Değişkenler

"Pclass","Embarked","Sex" sütunları kategorik -> onları encode etmemiz lazım

In [ ]:
train_data.head()

In [ ]:
# get_dummies() ile encode edelim

train_data = pd.get_dummies(train_data, columns=["Pclass","Embarked","Sex"], drop_first=True)

In [ ]:
train_data.head()

"PassengerId", "Name" ve "Ticket" sütunlarını atalım.

In [ ]:
train_data.drop('PassengerId', axis=1, inplace=True)
train_data.drop('Name', axis=1, inplace=True)
train_data.drop('Ticket', axis=1, inplace=True)

In [ ]:
train_data.head()

### 4. Exploratory Data Analysis (EDA)

In [ ]:
# train datanın şekli

train_data.shape

In [ ]:
# train data içindeki toplam veri adedi

print("train data içindeki toplam veri adedi:", train_data.shape[0])

In [ ]:
# sütun adları -> train

col_names = train_data.columns

col_names

#### 4.1 Age için EDA

In [ ]:
plt.figure(figsize=(15,8))

# Hayatta kalanlar -> Survived == 1
# https://seaborn.pydata.org/generated/seaborn.kdeplot.html
ax = sns.kdeplot(train_data["Age"][train_data.Survived == 1], color="green", shade=True)

# Ölenler -> Survived == 0
sns.kdeplot(train_data["Age"][train_data.Survived == 0], color="red", shade=True)

plt.legend(['Survived', 'Died'])
plt.title('Yaş (Age) için Hayatta Kalma ve Ölüm Yoğunluk Grafiği')
ax.set(xlabel='Age')
plt.xlim(-10,85)
plt.show()


Hayatta kalanların ve ölenlerin yaş dağılımları oldukça benzer.

Şunu söyleyebiliriz, hayatta kalanlar içinde çocuk ve genç yaşta olanların oranları daha fazla.

Bunun sebebi, ebevenylerin kendi hayatlarını kurtarmak yerine çocukların hayatlarını kurtarmak için onlara can yeleği ve kayık vermeleri olabilir. (sadece varsayım tabi)

#### 4.2 Fare (Ücret) için EDA

In [ ]:
plt.figure(figsize=(15,8))

ax = sns.kdeplot(train_data["Fare"][train_data.Survived == 1], color="green", shade=True)

sns.kdeplot(train_data["Fare"][train_data.Survived == 0], color="red", shade=True)

plt.legend(['Survived', 'Died'])
plt.title('Density Plot of Fare for Surviving Population and Deceased Population')
ax.set(xlabel='Fare')
plt.xlim(-20,200)
plt.show()

Ücret üzerinden baktığımızda hayatta kalma ve ölme oranlarının net olarak birbirinden ayrıldığını görüyoruz.

Ücretin bizim modelimizde önemli bir etken olma olasılığı yüksek.

Düşük ücret ödeyen yolcuların ölüm oranları çok daha yüksek.

Yüksek ücret ödeyenlerin hayatta kalma oranı daha fazla.

#### 4.3 Passanger Class (Yolcu Sınıfı) için EDA

In [ ]:
sns.barplot('Pclass', 'Survived', data=train_df, color="green")

plt.show()

Passenger Class'ı yani Yolcu Sınıfı 1. sınıf olan yolcuların hayatta kalma oranları daha yüksek.

Dolayısı ile bu da Ücret için yaptığımız varsayımı doğrular nitelikte.

#### 4.4 Aile ile veya Yalnız Seyahat için EDA

In [ ]:
sns.barplot('YalnizSeyahat', 'Survived', data=train_data, color="green")

plt.show()

Aile ile seyhat edenlerin (YalnizSeyahat = 0) hayatta kalma oranları daha yüksek.

Bu da bunların çoğunun çocuk ya da genç olduğu ve onların görece daha fazla hayatta kaldıkları varsayımımızı destekliyor.

#### 4.5 Sex (Cinsiyet) için EDA

In [ ]:
sns.barplot('Sex', 'Survived', data=train_df, color="green")

plt.show()

Cinsiyete baktığımızda Kadın ve Erkek'lerin hayatta kalma oranları arasında büyük bir fark görüyoruz.

Kadınların hayatta kalma oranları çok daha yüksek.

### 5. Input-Output Ayrımı

In [ ]:
y = train_data['Survived']

In [ ]:
train_data.drop('Survived', axis=1, inplace=True)

### 6. Feature Scaling (Boyutlama)

In [ ]:
# train datayı görelim

train_data.describe()

In [ ]:
cols = train_data.columns

cols

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# scaler'ı train data üzerinde eğitelim ve 
# train datayı scale edelim
train_data = scaler.fit_transform(train_data)

In [ ]:
type(train_data)

In [ ]:
# datayı DataFrame yapalım tekrar

train_data = pd.DataFrame(train_data, columns=[cols])

In [ ]:
train_data.head()

### 7. Train-Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# datayı artık gerçek olarak train-test şeklinde ayırabiliriz
# test size: %20 olsun

X_train, X_test, y_train, y_test = train_test_split(train_data, y, test_size=0.2, random_state=2)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train

In [ ]:
y_test

### 8. Model Oluşturma

In [ ]:
# LogisticRegression'u import et

from sklearn.linear_model import LogisticRegression

In [ ]:
# modeli yarat

logreg = LogisticRegression(solver='liblinear', random_state=0)

In [ ]:
# train data üzerinde LogisticRegression'u eğit

logreg.fit(X_train, y_train)

### 9. Tahmin

In [ ]:
# test data ile tahmin yap

y_pred = logreg.predict(X_test)

y_pred

**predict_proba** metodu size olasılıkları verir:
* 0.5'in sütü ise sonuç -> o sınıftır
* 0.5'in altı ise sonuç -> diğer sınıftır

In [ ]:
# 0 sınıfı -> ölüm (Servived = 0)

logreg.predict_proba(X_test)[:,0]

In [ ]:
# 1 sınıfı -> hayatta kalma (Servived = 1)

logreg.predict_proba(X_test)[:,1]

### 10. Tahmin Kalitesini Ölçmek

In [ ]:
from sklearn.metrics import accuracy_score

print("Modelin Accuracy Score'u: {0:0.4f}".format(accuracy_score(y_test, y_pred)))